# Does My Design Meet Requirements?

**The old way:** Three engineers — requirements, architecture, CAD — meet for weeks to manually check that everything lines up.

**The new way:** Pull data from Istari, run automated checks, get a pass/fail report in seconds.

This notebook:
1. Navigates the Istari system hierarchy — two models tracked in one configuration
2. Extracts requirements and architecture from a SysML model (via SysGit)
3. Loads CAD results from an nTop wing run (aerodeck metrics)
4. Runs compliance checks — does the CAD meet the requirements?
5. Finds a failure, updates the requirement, and re-checks
6. Snapshots each milestone so the team can see what changed and when

See [`example-output/`](example-output/) for pre-computed results.

In [ ]:
# Setup
import sys, json
from pathlib import Path

try:
    import istari_digital_client
except ImportError:
    !pip install istari-digital-client python-dotenv -q

repo_root = str(Path.cwd().parent.parent)
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from istari_client import get_client

client = get_client()
user = client.get_current_user()
print(f"Connected as: {user.display_name} ({user.email})")

In [ ]:
# Explore the system — navigate the Istari hierarchy
#
# This system tracks TWO models in one configuration:
#   - SysML model (requirements + architecture)
#   - nTop model (parametric wing CAD)
#
# Both are tracked as LATEST — any new revisions are automatically picked up.
#
# Istari links:
#   System: https://demo.istari.app/systems/294568b3-e626-4293-8e2c-307370ec9e95

SYSTEM_ID = "294568b3-e626-4293-8e2c-307370ec9e95"  # Example: Check Design Meets Requirements
CONFIG_ID = "cfdbd13b-817e-4c7e-b3c7-53ffcb4b9836"  # Baseline configuration

system = client.get_system(SYSTEM_ID)
print(f"System: {system.name}")
print(f"  {system.description}\n")

# Walk the hierarchy: configurations -> tracked files -> snapshots -> files
configs = client.list_system_configurations(SYSTEM_ID, page=1, size=50)
print(f"Configurations ({configs.total}):\n")

for config in configs.items:
    print(f"  {config.name}")
    print(f"    Config ID: {config.id}")

    # What files does this configuration track?
    tracked = client.list_tracked_files(config.id, page=1, size=50)
    print(f"    Tracked files ({tracked.total}):")
    for tf in tracked.items:
        mode = tf.specifier_type.value  # "Latest" or "Locked"
        print(f"      {tf.file_id} ({mode})")

    # What snapshots exist? (point-in-time captures)
    snapshots = client.list_snapshots(configuration_id=config.id, page=1, size=10)
    print(f"    Snapshots ({snapshots.total}):")

    for snap in snapshots.items:
        tags = client.list_tags(snapshot_id=snap.id, page=1, size=10)
        tag_names = [t.tag for t in tags.items]
        tag_str = f"  [{', '.join(tag_names)}]" if tag_names else ""
        revs = client.list_snapshot_revisions(snap.id, page=1, size=50)
        print(f"      {snap.id[:8]}...{tag_str}  ({revs.total} file(s))")
        for r in revs.items:
            size_kb = r.size / 1024
            if size_kb > 1024:
                print(f"        - {r.name} ({size_kb/1024:.1f} MB)")
            else:
                print(f"        - {r.name} ({size_kb:.1f} KB)")

In [ ]:
# Milestone 1: Load everything from Istari
#
# Two models, one system:
#   - SysML model → extract to get requirements JSON + parts JSON
#   - nTop model  → load aerodeck metrics from latest run
#
# Both are tracked in the same configuration as LATEST —
# any new revision is automatically picked up by the next snapshot.
#
# Istari links:
#   System:     https://demo.istari.app/systems/294568b3-e626-4293-8e2c-307370ec9e95
#   SysML model: https://demo.istari.app/files/c4280a27-b2e4-4376-81f7-474062bcdf4d
#   nTop model:  https://demo.istari.app/files/263b7332-03f4-4ded-9686-7f11df478058

SYSML_MODEL_ID = "c4280a27-b2e4-4376-81f7-474062bcdf4d"  # Group3 UAS Requirements
NTOP_MODEL_ID = "263b7332-03f4-4ded-9686-7f11df478058"    # Group3-UAS-Wing-v8

sysml_model = client.get_model(SYSML_MODEL_ID)
ntop_model = client.get_model(NTOP_MODEL_ID)

# Show both models with their revision history
for label, m in [("SysML", sysml_model), ("nTop", ntop_model)]:
    name = m.display_name or m.file.revisions[0].name
    print(f"{label} model: {name}")
    print(f"  File ID:    {m.file.id}")
    print(f"  Revisions:  {len(m.file.revisions)}")
    for rev in m.file.revisions:
        size_kb = rev.size / 1024
        ver = f"  ({rev.version_name})" if hasattr(rev, "version_name") and rev.version_name else ""
        print(f"    {rev.name} — {size_kb:.1f} KB{ver}")
    print(f"  Artifacts:  {len(m.artifacts)}")
    print()

# These models are tracked in the configuration we explored above
print(f"Both models tracked in config: {CONFIG_ID[:8]}...")
print("Tracked as LATEST — new revisions auto-included in next snapshot\n")

# Load the extracted requirements and parts
reqs_data = parts_data = metrics_data = None

for a in sysml_model.artifacts:
    rev = a.file.revisions[0] if a.file.revisions else None
    if rev and "requirements" in rev.name and rev.name.endswith(".json"):
        reqs_data = json.loads(a.read_text())
    elif rev and "parts" in rev.name and rev.name.endswith(".json"):
        parts_data = json.loads(a.read_text())

for a in reversed(ntop_model.artifacts):
    rev = a.file.revisions[0] if a.file.revisions else None
    if rev and "aerodeck_metrics" in rev.name:
        metrics_data = json.loads(a.read_text())
        break

print(f"Loaded: {len(reqs_data)} requirements, {len(parts_data)} parts, aerodeck metrics")
print("\n--- Everything in one place. Ready to check. ---")

In [ ]:
# Quick look: what are the targets and actuals?
from IPython.display import HTML, display

range_info = metrics_data["range_mission"]
mass_info = metrics_data["mass_properties"]

html = """
<table style="border-collapse: collapse; font-size: 14px;">
<thead>
    <tr style="background: #f1f5f9;">
        <th style="padding: 8px 16px; text-align: left; border-bottom: 2px solid #cbd5e1;">Metric</th>
        <th style="padding: 8px 16px; text-align: left; border-bottom: 2px solid #cbd5e1;">Requirement</th>
        <th style="padding: 8px 16px; text-align: left; border-bottom: 2px solid #cbd5e1;">Actual (CAD)</th>
    </tr>
</thead>
<tbody>
    <tr><td style="padding: 6px 16px;">Range</td><td>\u2265 1,500 nm</td><td><b>""" + f"{range_info['range_nm']:,.0f} nm" + """</b></td></tr>
    <tr><td style="padding: 6px 16px;">Structure Weight</td><td>\u2264 275 lb</td><td><b>""" + f"{mass_info['empty_weight_lbm']} lb" + """</b></td></tr>
    <tr><td style="padding: 6px 16px;">Cruise Speed</td><td>\u2265 100 kts</td><td><b>""" + f"{range_info['cruise_speed_kts']} kts" + """</b></td></tr>
</tbody>
</table>"""
display(HTML(html))

In [ ]:
# Milestone 2: Run compliance checks
from compliance_checks import run_all_checks, format_report

results = run_all_checks(reqs_data, parts_data, metrics_data)
print(format_report(results))

# Show as HTML table
rows = ""
for r in results:
    color = "#059669" if r["status"] == "PASS" else "#dc2626"
    icon = "\u2705" if r["status"] == "PASS" else "\u274c"
    margin = f"+{r['margin']}%" if r["margin"] >= 0 else f"{r['margin']}%"
    rows += f'<tr><td style="padding: 6px 12px;">{r["check"]}</td>'
    rows += f'<td style="padding: 6px 12px;">{r["target"]} {r["unit"]}</td>'
    rows += f'<td style="padding: 6px 12px;">{r["actual"]} {r["unit"]}</td>'
    rows += f'<td style="padding: 6px 12px; color: {color}; font-weight: bold;">{icon} {r["status"]} ({margin})</td></tr>\n'

html = f"""
<table style="border-collapse: collapse; font-size: 14px; margin-top: 12px;">
<thead>
    <tr style="background: #f1f5f9;">
        <th style="padding: 8px 12px; text-align: left; border-bottom: 2px solid #cbd5e1;">Check</th>
        <th style="padding: 8px 12px; text-align: left; border-bottom: 2px solid #cbd5e1;">Target</th>
        <th style="padding: 8px 12px; text-align: left; border-bottom: 2px solid #cbd5e1;">Actual</th>
        <th style="padding: 8px 12px; text-align: left; border-bottom: 2px solid #cbd5e1;">Result</th>
    </tr>
</thead>
<tbody>{rows}</tbody>
</table>"""
display(HTML(html))

In [ ]:
# Snapshot: capture state after initial compliance checks
#
# Weight FAILS — but before we change anything, snapshot the current state.
# This gives us a "before" reference point for the requirement update.

from istari_digital_client import NewSnapshot, NewSnapshotTag

print("Creating snapshot: initial-checks...")
snap_response = client.create_snapshot(CONFIG_ID, NewSnapshot())
snapshot = snap_response.actual_instance

# Handle NoOp (no files changed since last snapshot)
if hasattr(snapshot, "id"):
    snap_id = snapshot.id
    print(f"  New snapshot: {snap_id[:8]}...")
else:
    # No changes since last snapshot — tag the most recent one
    snaps = client.list_snapshots(configuration_id=CONFIG_ID, page=1, size=1)
    snap_id = snaps.items[0].id
    print(f"  No changes — tagging existing snapshot: {snap_id[:8]}...")

client.create_tag(snap_id, NewSnapshotTag(tag="initial-checks"))
print("  Tagged: initial-checks")

revs = client.list_snapshot_revisions(snap_id, page=1, size=50)
print(f"  Files: {revs.total}")
print("\n--- Baseline captured. Now let's fix the weight requirement. ---")

In [ ]:
# Milestone 3: Weight fails — update the requirement and re-check
#
# The wing weighs 321.8 lb but the requirement says 275 lb.
# The team agrees to relax the weight budget to 325 lb.
# Update the SysML, re-upload, re-extract, re-check.

import tempfile

print("Structure Weight: 321.8 lb vs 275 lb requirement --> FAIL")
print("Team decision: relax weight budget to 325 lb\n")

# Download current SysML, update the requirement
sysml_text = sysml_model.read_text()
updated_text = sysml_text.replace(
    'attribute maxValue : Real = 275.0;',
    'attribute maxValue : Real = 325.0;'
).replace(
    'shall not exceed 275 lb',
    'shall not exceed 325 lb'
)

# Verify the change
changes = 0
for old, new in zip(sysml_text.splitlines(), updated_text.splitlines()):
    if old != new:
        changes += 1
        print(f"  - {old.strip()}")
        print(f"  + {new.strip()}")
        print()
print(f"{changes} lines changed")

# Write to temp file and upload as new revision
with tempfile.NamedTemporaryFile(mode="w", suffix=".sysml", delete=False) as tmp:
    tmp.write(updated_text)
    tmp_path = Path(tmp.name)

client.update_model(model_id=SYSML_MODEL_ID, path=tmp_path)
tmp_path.unlink()  # clean up temp file
print("Uploaded new revision to Istari")

In [ ]:
# Re-extract requirements and re-run checks
from time import sleep
from datetime import datetime
from istari_digital_client import JobStatusName

print("Re-extracting requirements...")
job = client.add_job(
    model_id=SYSML_MODEL_ID,
    function="@istari:extract_sysmlv2",
    tool_name="sysgit",
    tool_version="0.1.8",
    operating_system="Ubuntu 22.04",
    parameters={},
)
print(f"Job: {job.id}")

while True:
    sleep(5)
    job = client.get_job(job.id)
    ts = datetime.now().strftime("%H:%M:%S")
    status = job.status.name.value
    print(f"  [{ts}] {status}")
    if job.status.name in {JobStatusName.COMPLETED, JobStatusName.FAILED}:
        break

if job.status.name == JobStatusName.COMPLETED:
    # Reload updated requirements
    sysml_model = client.get_model(SYSML_MODEL_ID)
    for a in sysml_model.artifacts:
        rev = a.file.revisions[0] if a.file.revisions else None
        if rev and "requirements" in rev.name and rev.name.endswith(".json"):
            reqs_data = json.loads(a.read_text())
            break

    # Re-run checks with updated requirements
    print("\n--- Updated compliance report ---\n")
    results = run_all_checks(reqs_data, parts_data, metrics_data)
    print(format_report(results))

    # Show updated HTML table
    rows = ""
    for r in results:
        color = "#059669" if r["status"] == "PASS" else "#dc2626"
        icon = "\u2705" if r["status"] == "PASS" else "\u274c"
        margin = f"+{r['margin']}%" if r["margin"] >= 0 else f"{r['margin']}%"
        rows += f'<tr><td style="padding: 6px 12px;">{r["check"]}</td>'
        rows += f'<td style="padding: 6px 12px;">{r["target"]} {r["unit"]}</td>'
        rows += f'<td style="padding: 6px 12px;">{r["actual"]} {r["unit"]}</td>'
        rows += f'<td style="padding: 6px 12px; color: {color}; font-weight: bold;">{icon} {r["status"]} ({margin})</td></tr>\n'

    html = f"""
    <table style="border-collapse: collapse; font-size: 14px; margin-top: 12px;">
    <thead>
        <tr style="background: #f1f5f9;">
            <th style="padding: 8px 12px; text-align: left; border-bottom: 2px solid #cbd5e1;">Check</th>
            <th style="padding: 8px 12px; text-align: left; border-bottom: 2px solid #cbd5e1;">Target</th>
            <th style="padding: 8px 12px; text-align: left; border-bottom: 2px solid #cbd5e1;">Actual</th>
            <th style="padding: 8px 12px; text-align: left; border-bottom: 2px solid #cbd5e1;">Result</th>
        </tr>
    </thead>
    <tbody>{rows}</tbody>
    </table>"""
    display(HTML(html))
else:
    print("Extraction failed!")

In [ ]:
# Snapshot: capture state after requirement update + re-extraction
#
# The SysML model now has revision 2 (325 lb) and fresh extraction artifacts.
# Snapshot this milestone so the team can see what changed.

print("Creating snapshot: post-requirement-update...")
snap_response = client.create_snapshot(CONFIG_ID, NewSnapshot())
snapshot = snap_response.actual_instance

if hasattr(snapshot, "id"):
    snap_id = snapshot.id
    print(f"  New snapshot: {snap_id[:8]}...")
else:
    snaps = client.list_snapshots(configuration_id=CONFIG_ID, page=1, size=1)
    snap_id = snaps.items[0].id
    print(f"  No changes — tagging existing snapshot: {snap_id[:8]}...")

client.create_tag(snap_id, NewSnapshotTag(tag="post-requirement-update"))
print("  Tagged: post-requirement-update")

revs = client.list_snapshot_revisions(snap_id, page=1, size=50)
print(f"  Files: {revs.total}")

# Show what's new in this snapshot
print("\n  Files in this snapshot:")
for r in revs.items:
    size_kb = r.size / 1024
    print(f"    {r.name} ({size_kb:.1f} KB)")

In [ ]:
# View system state — all snapshots with tags, files, and sizes
#
# After the full workflow, our configuration should have 4 tagged snapshots:
#   1. initial-upload / baseline  — raw models, no artifacts
#   2. post-extraction           — models + extraction & nTop artifacts
#   3. initial-checks            — same state (compliance checks don't create files)
#   4. post-requirement-update   — SysML rev 2 (325 lb) + new extraction artifacts

print(f"System: {system.name}")
print(f"  {system.description}\n")

configs = client.list_system_configurations(SYSTEM_ID, page=1, size=50)
for config in configs.items:
    print(f"Configuration: {config.name}")
    print(f"  ID: {config.id}")

    tracked = client.list_tracked_files(config.id, page=1, size=50)
    print(f"  Tracked files: {tracked.total}")
    for tf in tracked.items:
        mode = tf.specifier_type.value
        print(f"    {tf.file_id} ({mode})")

    snapshots = client.list_snapshots(configuration_id=config.id, page=1, size=20)
    print(f"\n  Snapshots ({snapshots.total}):\n")

    for i, snap in enumerate(snapshots.items):
        tags = client.list_tags(snapshot_id=snap.id, page=1, size=10)
        tag_names = [t.tag for t in tags.items]
        tag_str = ", ".join(tag_names) if tag_names else "untagged"
        revs = client.list_snapshot_revisions(snap.id, page=1, size=50)

        print(f"    {i+1}. [{tag_str}]  ({revs.total} files)")
        for r in revs.items:
            size_kb = r.size / 1024
            if size_kb > 1024:
                print(f"       {r.name} ({size_kb/1024:.1f} MB)")
            else:
                print(f"       {r.name} ({size_kb:.1f} KB)")
        print()

## Summary

**What we did:**
1. Navigated the Istari system hierarchy — two models tracked in one configuration
2. Pulled requirements (SysML) and CAD results (nTop aerodeck) from Istari
3. Ran automated compliance checks — found that weight **fails** (321.8 lb vs 275 lb)
4. Snapshotted the "initial-checks" state before making changes
5. Updated the requirement (275 → 325 lb), re-extracted, re-checked — **all green**
6. Snapshotted the "post-requirement-update" state with the fix

### Version History

| # | Snapshot Tag | Files | What happened |
|---|-------------|-------|---------------|
| 1 | `initial-upload` / `baseline` | 2 | Raw SysML + nTop models uploaded |
| 2 | `post-extraction` | ~20 | SysGit + nTop extraction artifacts added |
| 3 | `initial-checks` | ~20 | Compliance checks run — weight FAILS |
| 4 | `post-requirement-update` | ~20+ | SysML rev 2 (325 lb) + new extraction artifacts |

### What Istari Did

| Step | Inner Loop (Tools) | Outer Loop (Istari) |
|------|-------------------|---------------------|
| Upload models | — | Stored SysML + nTop files, tracked as LATEST |
| Extract requirements | SysGit parsed SysML v2 | Ran job, stored JSON + PNG artifacts |
| Run nTop | nTop generated aerodeck metrics | Ran job, stored 14 output artifacts |
| Compliance checks | Python script compared reqs vs CAD | — (ran locally) |
| Update requirement | — | New file revision (275→325 lb), versioned |
| Re-extract | SysGit re-parsed updated SysML | Ran job, stored updated artifacts |
| Snapshot milestones | — | 4 snapshots capturing each milestone |

### Old Way vs New Way

**Old way:** Weeks of meetings between requirements, architecture, and CAD engineers to manually check alignment.

**New way:** Everyone works in their own tool. Istari stores everything. Automated checks run in seconds. When something changes, re-check immediately. Snapshots track every milestone.

### What's Next?

- **Add more checks** — extend `compliance_checks.py` with your own project-specific rules
- **Run in CI** — trigger checks automatically when any model is updated in Istari
- **Compare across runs** — check compliance for different nTop parameter sets to find the best design
- **Compare snapshots** — see what changed between milestones using the version history